In [1]:
%config Completer.use_jedi = False
import warnings

warnings.filterwarnings(action='ignore')

In [2]:
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


퍼셉트론은 엄밀히 말하면 딥러닝은 아니지만 뉴런과 상당히 닮은 분류기이다.
뉴런의 계산 과정을 이해하고 뉴런 하나만으로 AND, OR 연산과 같은 기초적인 연산을 할 수 있음을 알아본다.

In [3]:
# 퍼셉트론 코드의 이해도를 높이기 위해 1은 True로 0은 False로 그리고 편향(bias)은 1로 지정한다.
T = 1.0
F = 0.0
bias = 1.0

AND, OR, XOR 실습 데이터를 함수 형태로 정의해서 함수를 호출해서 데이터를 획득할 수 있게 한다.

In [4]:
def get_AND_data():
    X = [
        [F, F, bias],
        [F, T, bias],
        [T, F, bias],
        [T, T, bias]
    ]  # 학습 데이터
    Y = [
        [F],
        [F],
        [F],
        [T]
    ]  # 레이블, 클래스, 타겟, 실제값
    return X, Y


def get_OR_data():
    X = [[F, F, bias], [F, T, bias], [T, F, bias], [T, T, bias]]
    Y = [[F], [T], [T], [T]]
    return X, Y


def get_XOR_data():
    X = [[F, F, bias], [F, T, bias], [T, F, bias], [T, T, bias]]
    Y = [[F], [T], [T], [F]]
    return X, Y

In [5]:
# AND 연산에 필요한 데이터를 호출한다.
X, Y = get_AND_data()
# OR 연산에 필요한 데이터를 호출한다.
# X, Y = get_OR_data()
# XOR 연산에 필요한 데이터를 호출한다.
# X, Y = get_XOR_data()
X, Y

([[0.0, 0.0, 1.0], [0.0, 1.0, 1.0], [1.0, 0.0, 1.0], [1.0, 1.0, 1.0]],
 [[0.0], [0.0], [0.0], [1.0]])

텐서플로우를 사용해서 퍼셉트론을 구현한다.

In [6]:
# 논리 연산을 위한 입력값으로 X, Y, 편향값을 입력받아(총 3개의 입력) 1개의 값을 출력해야 하므로 가중치는 [3, 1]로 설정한다.
W = tf.Variable(tf.random_normal([3, 1]))  # matmul() 함수로 행렬의 곱셈을 실행하기 위해 3행 1열로 가중치를 만든다.
sess = tf.Session()  # 세션을 만든다.
sess.run(tf.global_variables_initializer())  # 변수를 초기화시킨다.
print(sess.run(W))

Metal device set to: Apple M2


2023-03-02 19:29:04.153660: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-02 19:29:04.154173: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-03-02 19:29:04.163043: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:357] MLIR V1 optimization pass is not enabled
2023-03-02 19:29:04.170772: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-03-02 19:29:04.177749: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


[[-1.368712  ]
 [ 0.60130924]
 [ 0.9728969 ]]


2023-03-02 19:29:04.497886: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


퍼셉트론은 두 개의 입력이 있을 때 하나의 뉴런으로 두 개의 입력을 계산한 뒤 최종 결과값으로 0 또는 1을 출력한다.
두 개의 입력은 가중치와 곱해져서 뉴런의 첫 번째 단계인 시그마로 들어가고 시그마 단계에서 모든 가중치 * 입력값과 편향값을 더하는 과정이 이루어진다.
시그마 값은 뉴런의 두 번째 단계은 활성화 함수의 입력으로 들어가는데 퍼셉트론 스텝 함수라는 활성화 함수를 사용한다. 스텝 함수는 시그마 값 >= 0 일 경우 1, 시그마 값 < 0 일 경우 0을 출력한다.

In [7]:
# x가 0 이상이면 1, x가 0보다 작으면 0을 리턴하는 활성화 함수(스탭 함수)를 구현한다.
# tf.greater(a, b): a > b일 경우 True, 그렇치 않으면 False를 리턴한다.
# tf.greater_equal(a, b): a >= b일 경우 True, 그렇치 않으면 False를 리턴한다.
# tf.less(a, b): a < b일 경우 True, 그렇치 않으면 False를 리턴한다.
# tf.less_equal(a, b): a <= b일 경우 True, 그렇치 않으면 False를 리턴한다.
# tf.equal(a, b): a = b일 경우 True, 그렇치 않으면 False를 리턴한다.
# tf.not_equal(a, b): a <> b일 경우 True, 그렇치 않으면 False를 리턴한다.
sess = tf.Session()
print(sess.run(tf.greater_equal(1, 0)))
f = tf.cast(tf.greater_equal(1, 0), dtype=tf.float32)
print(sess.run(f))

True
1.0


2023-03-02 19:29:11.206209: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-02 19:29:11.206239: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-03-02 19:29:11.211149: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-03-02 19:29:11.217150: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [8]:
def step(x):
    return tf.cast(tf.greater_equal(x, 0), dtype=tf.float32)

손실(오차, 비용) 함수로 평균 제곱 오차(MSE)를 사용한다.

In [9]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
f = tf.matmul(X, W)  # 행렬 곱셈 함수로 데이터와 가중치를 곱한다.
# print(sess.run(f))
output = step(f)  # 스텝 함수를 실행한다. => 예측값
# print(sess.run(output))
error = tf.subtract(Y, output)  # 레이블과(실제값) 스텝 함수의 실행 결과(예측값)의 편차를 계산한다.
# print(sess.run(error))
mse = tf.reduce_mean(tf.square(error))  # 평균 제곱 오차를 계산한다.
# print(sess.run(mse))

2023-03-02 19:29:27.343667: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-02 19:29:27.343688: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-03-02 19:29:27.346847: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


보통의 경우 딥러닝 모델은 경사 하강법을 통해 매개 변수를 최적화하지만 미분값이 큰 의미가 없는 스텝 함수의 영향으로(입력값이 0일 때 미분 불가능, 입력값이 1일 때 미분값은 항상 0) 퍼셉트론은 아래와 같은 방법으로 최적화를 진행한다.
***
결과값이 1이 나와야 하는데 스텝 함수의 출력이 0일 경우 => w_new = w_old + delta
결과값이 0이 나와야 하는데 스텝 함수의 출력이 1일 경우 => w_new = w_old - delta
위의 2가지 경우를 다음과 같이 사용할 수 있다.

In [10]:
# 행렬의 transpose는 행과 열을 교환하여 얻는 전치행렬을 의미한다.
# matmul() 함수의 transpose_a=True 옵션을 사용하면 행렬을 곱하기 전에 1번째 행렬을 전치하고 transpose_b=True 옵션을 사용하면
# 행렬을 곱하기 전에 2번째 행렬을 전치해서 곱셈한다.
# import numpy as np
# print(type(X))
# print(type(np.array(X)))
# print(np.array(X))
# print(np.array(X).T)
delta = tf.matmul(X, error, transpose_a=True)
# print(sess.run(delta))
# assign() 함수는 세션 안에서 텐서플로우 변수 값을 변경한다.
train = tf.assign(W, tf.add(W, delta))  # 역전파
# print(sess.run(train))

학습 후 테스트

In [11]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

err, epoch, max_epochs = 1.0, 0, 20  # 오차, 학습 횟수, 최대 학습 회수
# 오차가 발생하고 학습 횟수가 최대 학습 회수를 넘어가지 않는동안 반복한다.
while err > 0.0 and epoch < max_epochs:
    epoch += 1  # 학습 횟수를 증가시킨다.
    err = sess.run(mse)  # 평균 제곱 오차를 계산한다.
    sess.run(train)  # 학습시킨다.
    print('epoch: {:2d}, mse: {:4.2f}'.format(epoch, err))
# ===== while

print('테스트 결과\n', sess.run(output), sep='')
sess.close()

2023-03-02 19:31:42.522920: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-02 19:31:42.522979: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-03-02 19:31:42.530275: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-03-02 19:31:42.539391: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


epoch:  1, mse: 0.50
epoch:  2, mse: 0.25
epoch:  3, mse: 0.25
epoch:  4, mse: 0.50
epoch:  5, mse: 0.25
epoch:  6, mse: 0.00
테스트 결과
[[0.]
 [0.]
 [0.]
 [1.]]


2023-03-02 19:31:42.837251: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-03-02 19:31:42.934782: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [12]:
with tf.Session() as sess:  # 세션을 만든다.
    sess.run(tf.global_variables_initializer())  # 텐서플로우 변수를 초기화한다.
    err, epoch, max_epochs = 1.0, 0, 20
    while err > 0.0 and epoch < max_epochs:
        epoch += 1
        err = sess.run(mse)
        sess.run(train)
        print('epoch: {:2d}, mse: {:4.2f}'.format(epoch, err))
    # ===== while
    print('테스트 결과\n', sess.run(output), sep='')

epoch:  1, mse: 0.75
epoch:  2, mse: 0.25
epoch:  3, mse: 0.25
epoch:  4, mse: 0.25
epoch:  5, mse: 0.50
epoch:  6, mse: 0.25
epoch:  7, mse: 0.00
테스트 결과
[[0.]
 [0.]
 [0.]
 [1.]]


2023-03-02 19:31:56.886231: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-02 19:31:56.886266: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-03-02 19:31:56.888948: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-03-02 19:31:56.896380: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-03-02 19:31:56.909438: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-03-02 19:31:56.93